In [1]:
import numpy as np
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import nltk
import spacy
from nltk.corpus import stopwords,wordnet
from nltk.tokenize import word_tokenize, RegexpTokenizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import string
import re
from sklearn.feature_extraction import text 
%matplotlib inline 

In [2]:
data = pd.read_csv("blogtext.csv")

In [3]:
data.columns


Index(['id', 'gender', 'age', 'topic', 'sign', 'date', 'text'], dtype='object')

In [4]:
data.shape

(681284, 7)

In [5]:
data['text']

0                    Info has been found (+/- 100 pages,...
1                    These are the team members:   Drewe...
2                    In het kader van kernfusie op aarde...
3                          testing!!!  testing!!!          
4                      Thanks to Yahoo!'s Toolbar I can ...
                                ...                        
681279           Dear Susan,  I could write some really ...
681280           Dear Susan,  'I have the second yeast i...
681281           Dear Susan,  Your 'boyfriend' is fuckin...
681282           Dear Susan:    Just to clarify, I am as...
681283           Hey everybody...and Susan,  You might a...
Name: text, Length: 681284, dtype: object

In [6]:
new_data = data.head(9000)

In [7]:
new_data['label'] = new_data[new_data.columns[1:5]].apply(lambda x: (','.join(x.dropna().astype(str))),axis=1)

C:\Users\pg\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
new_data

id gender  age              topic      sign          date  \
0     2059027   male   15            Student       Leo   14,May,2004   
1     2059027   male   15            Student       Leo   13,May,2004   
2     2059027   male   15            Student       Leo   12,May,2004   
3     2059027   male   15            Student       Leo   12,May,2004   
4     3581210   male   33  InvestmentBanking  Aquarius  11,June,2004   
...       ...    ...  ...                ...       ...           ...   
8995  3477296   male   15            Student     Virgo  17,July,2004   
8996  3477296   male   15            Student     Virgo  17,July,2004   
8997  3477296   male   15            Student     Virgo  17,July,2004   
8998  3477296   male   15            Student     Virgo  16,July,2004   
8999  3477296   male   15            Student     Virgo  16,July,2004   

                                                   text  \
0                Info has been found (+/- 100 pages,...   
1                These are the team members:   Drewe...   
2                In het kader van kernfusie op aarde...   
3                      testing!!!  testing!!!             
4                  Thanks to Yahoo!'s Toolbar I can ...   
...                                                 ...   
8995                   wah! how cold was it today? *...   
8996                   Could someone give me some su...   
8997                   I am blogging here tonight on...   
8998                   'It isn't pollution that's ha...   
8999                   Seems like everyone these day...   

                                   label  
0                    male,15,Student,Leo  
1                    male,15,Student,Leo  
2                    male,15,Student,Leo  
3                    male,15,Student,Leo  
4     male,33,InvestmentBanking,Aquarius  
...                                  ...  
8995               male,15,Student,Virgo  
8996               male,15,Student,Virgo  
8997               male,15,Student,Virgo  
8998               male,15,Student,Virgo  
8999               male,15,Student,Virgo  

[9000 rows x 8 columns]

In [9]:
new_data = new_data.drop(columns=['gender','age','topic','sign','id','date'])

In [10]:
new_data

text  \
0                Info has been found (+/- 100 pages,...   
1                These are the team members:   Drewe...   
2                In het kader van kernfusie op aarde...   
3                      testing!!!  testing!!!             
4                  Thanks to Yahoo!'s Toolbar I can ...   
...                                                 ...   
8995                   wah! how cold was it today? *...   
8996                   Could someone give me some su...   
8997                   I am blogging here tonight on...   
8998                   'It isn't pollution that's ha...   
8999                   Seems like everyone these day...   

                                   label  
0                    male,15,Student,Leo  
1                    male,15,Student,Leo  
2                    male,15,Student,Leo  
3                    male,15,Student,Leo  
4     male,33,InvestmentBanking,Aquarius  
...                                  ...  
8995               male,15,Student,Virgo  
8996               male,15,Student,Virgo  
8997               male,15,Student,Virgo  
8998               male,15,Student,Virgo  
8999               male,15,Student,Virgo  

[9000 rows x 2 columns]

In [11]:
print(len(new_data.text))

9000


In [12]:
new_data.text[0]

'           Info has been found (+/- 100 pages, and 4.5 MB of .pdf files) Now i have to wait untill our team leader has processed it and learns html.         '

In [13]:
nlp = spacy.load('en_core_web_sm')
l1 = ('btw','zza','zzzexy','zzzzz','youuuuu')
nlp.Defaults.stop_words.add(l1)


In [14]:
#for i in range(len(new_data.text)):
 #   new_data.text[i] = new_data.text[i].lower()

In [15]:
new_data.text

0                  Info has been found (+/- 100 pages,...
1                  These are the team members:   Drewe...
2                  In het kader van kernfusie op aarde...
3                        testing!!!  testing!!!          
4                    Thanks to Yahoo!'s Toolbar I can ...
                              ...                        
8995                     wah! how cold was it today? *...
8996                     Could someone give me some su...
8997                     I am blogging here tonight on...
8998                     'It isn't pollution that's ha...
8999                     Seems like everyone these day...
Name: text, Length: 9000, dtype: object

In [16]:
new_data.dtypes

text     object
label    object
dtype: object

In [19]:
for i in range(len(new_data.text)):
    tokenizer = RegexpTokenizer(r'\w+')
    new_data.text[i] = new_data.text[i].lower()
    word_tokens = tokenizer.tokenize(new_data.text[i])
    filtered_sentence = [w for w in word_tokens if not w in stopwords.words('english')] 
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in (nlp.Defaults.stop_words or string.punctuation):
            #if not w.isalpha():
            filtered_sentence.append(re.sub(r"[^a-zA-Z]+", ' ',w ))
   
    new_data.text[i] = " ".join(filtered_sentence)
#print(word_tokens) 
#    print(new_data.text_new[i])
%time

Wall time: 0 ns


In [20]:
for i in range(len(new_data.text)):
#    tokenizer = RegexpTokenizer(r'\w+')
 #   word_tokens = tokenizer.tokenize(new_data.text[i])
    new_data.text[i] = re.sub("\s\s+" , " ", new_data.text[i])
#print(word_tokens) 
#    print(new_data.text_new[i])
%time


Wall time: 0 ns


In [21]:
new_data.text

0       info found pages mb pdf files wait untill team...
1       team members drewes van der laag urllink mail ...
2       het kader van kernfusie op aarde maak je eigen...
3                                         testing testing
4       thanks yahoo s toolbar capture urls popups mea...
                              ...                        
8995    wah cold today brrr wake worth grins said slep...
8996    suggestions music dls im open asian music look...
8997    blogging tonight request andy know blog tonigh...
8998    isn t pollution s harming environment s impuri...
8999    like days turning religion times peril terror ...
Name: text, Length: 9000, dtype: object

In [22]:
X = new_data.text
Y = new_data.label

In [23]:

# split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=524)

In [24]:
# define a function that accepts a vectorizer and calculates the accuracy
def tokenize_test(vect):
    X_train_dtm = vect.fit_transform(X_train)
    print('Features: ', X_train_dtm.shape[1])
    X_test_dtm = vect.transform(X_test)
    nb = MultinomialNB()
    nb.fit(X_train_dtm, y_train)
    #feature = nb.feature_count_
    #print("NB feature shape", feature)
   # print(nb.feature_count_.shape)
    y_train_pred = nb.predict(X_train_dtm)
    y_pred_class = nb.predict(X_test_dtm)
    print('Train Accuracy for NB : ', metrics.accuracy_score(y_train,y_train_pred))
    print('Test Accuracy for NB: ', metrics.accuracy_score(y_test, y_pred_class))
    logreg = LogisticRegression(C=1e9)
    logreg.fit(X_train_dtm, y_train)
    y_pred_class_LR = logreg.predict(X_test_dtm)
#print(metrics.accuracy_score(y_test, y_pred_class))
    y_train_LR = logreg.predict(X_train_dtm)
    print('Train Accuracy for LR: ',metrics.accuracy_score(y_train, y_train_LR))
    print('Test Accuracy for LR: ',metrics.accuracy_score(y_test, y_pred_class_LR))
%time

Wall time: 0 ns


In [25]:
# include 1-grams and 2-grams
vect = CountVectorizer(ngram_range=(1, 2))
tokenize_test(vect)

Features:  411221
Train Accuracy for NB :  0.7666666666666667
Test Accuracy for NB:  0.3648888888888889


C:\Users\pg\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\pg\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Train Accuracy for LR:  0.9943703703703703
Test Accuracy for LR:  0.5066666666666667


In [26]:
X_train_dtm = vect.fit_transform(X_train)
print('Features: ', X_train_dtm.shape[1])
X_test_dtm = vect.transform(X_test)
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
    #feature = nb.feature_count_
    #print("NB feature shape", feature)
   # print(nb.feature_count_.shape)
y_train_pred = nb.predict(X_train_dtm)
y_pred_class = nb.predict(X_test_dtm)
print('Train Accuracy for NB : ', metrics.accuracy_score(y_train,y_train_pred))
print('Test Accuracy for NB: ', metrics.accuracy_score(y_test, y_pred_class))

Features:  411221
Train Accuracy for NB :  0.7666666666666667
Test Accuracy for NB:  0.3648888888888889


In [27]:
# features names
feature_names = vect.get_feature_names()
print(feature_names[50:100])

['aargh', 'aargh told', 'aaron', 'aaron aaron', 'aaron broke', 'aaron brown', 'aaron burr', 'aaron came', 'aaron club', 'aaron elandt', 'aaron gf', 'aaron heard', 'aaron heath', 'aaron hehe', 'aaron hi', 'aaron james', 'aaron know', 'aaron lacrosse', 'aaron like', 'aaron likes', 'aaron love', 'aaron making', 'aaron oppsy', 'aaron pak', 'aaron pops', 'aaron pulled', 'aaron realized', 'aaron saying', 'aaron seeing', 'aaron sill', 'aaron stephen', 'aaron talladega', 'aaron waited', 'aaron went', 'aarrgghh', 'aarrgghh missed', 'ab', 'ab hoffman', 'ab reduction', 'aba', 'aba game', 'aba sus', 'aback', 'aback bit', 'aback blog', 'aback catholicism', 'aback coz', 'aback sure', 'aballaby', 'aballaby nbsp']


In [28]:
y_train.head(4)

2122    male,35,Technology,Aries
5676     female,27,indUnk,Taurus
7396       male,36,Fashion,Aries
1668    male,35,Technology,Aries
Name: label, dtype: object

In [29]:
len(y_train)

6750

In [30]:
d = {}

In [31]:
d = y_train.apply(lambda x : pd.value_counts(x.split(","))).sum(axis = 0).to_dict()

In [32]:
d

{'Technology': 1970.0,
 'Aries': 3155.0,
 '35': 1753.0,
 'male': 4160.0,
 'indUnk': 2051.0,
 'Taurus': 588.0,
 '27': 665.0,
 'female': 2590.0,
 'Fashion': 1230.0,
 '36': 1298.0,
 'Pisces': 288.0,
 'Internet': 89.0,
 'Aquarius': 301.0,
 'InvestmentBanking': 54.0,
 '33': 102.0,
 '24': 346.0,
 'Engineering': 102.0,
 'Libra': 372.0,
 '17': 745.0,
 'Scorpio': 672.0,
 'Education': 184.0,
 '34': 414.0,
 '14': 133.0,
 'Student': 689.0,
 'Sagittarius': 664.0,
 '25': 270.0,
 'Cancer': 230.0,
 'Capricorn': 101.0,
 'Gemini': 107.0,
 '15': 416.0,
 '26': 143.0,
 'Leo': 163.0,
 '23': 156.0,
 'Marketing': 9.0,
 'Banking': 14.0,
 'BusinessServices': 62.0,
 'Sports-Recreation': 53.0,
 '16': 126.0,
 'Religion': 8.0,
 '41': 15.0,
 '13': 29.0,
 'Communications-Media': 76.0,
 '39': 60.0,
 'Virgo': 109.0,
 'Consulting': 10.0,
 'Law': 6.0,
 'Science': 45.0,
 '42': 9.0,
 'Automotive': 7.0,
 'Arts': 26.0,
 '38': 37.0,
 'Non-Profit': 57.0,
 '37': 13.0,
 '43': 4.0,
 'Accounting': 4.0,
 '45': 10.0,
 '44': 2.0,
 'M

In [33]:
from sklearn.preprocessing import MultiLabelBinarizer

In [34]:
mlb = MultiLabelBinarizer()

In [35]:
y_train_mlb = mlb.fit_transform(y_train)

In [36]:
y_test_mlb = mlb.transform(y_test)

In [37]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

In [44]:
LR = LogisticRegression(solver = 'lbfgs',random_state= 111,max_iter = 1000)

In [45]:
LR

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=111, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [46]:
clf = OneVsRestClassifier(LR)

In [47]:
names = vect.get_feature_names()

In [48]:
y_train_mlb

array([[1, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [49]:
clf.fit(X_train_dtm,y_train_mlb)
%time

C:\Users\pg\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label 0 is present in all training examples.
  str(classes[c]))
C:\Users\pg\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label 28 is present in all training examples.
  str(classes[c]))
C:\Users\pg\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label 32 is present in all training examples.
  str(classes[c]))
C:\Users\pg\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label 38 is present in all training examples.
  str(classes[c]))
C:\Users\pg\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label 39 is present in all training examples.
  str(classes[c]))


Wall time: 0 ns


In [50]:
y_pred_clf = clf.predict(X_test_dtm)

In [51]:
print(metrics.accuracy_score(y_test_mlb,y_pred_clf))

0.2617777777777778


In [52]:
from sklearn.metrics import classification_report

In [53]:
print(classification_report(y_test_mlb, y_pred_clf))

C:\Users\pg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\pg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2250
           1       0.71      0.19      0.30        64
           2       0.00      0.00      0.00         0
           3       0.86      0.46      0.60       542
           4       0.75      0.35      0.48       526
           5       0.80      0.83      0.81      1241
           6       0.88      0.36      0.51       331
           7       0.76      0.55      0.64       797
           8       0.88      0.47      0.62       498
           9       0.83      0.33      0.47       505
          10       0.75      0.33      0.46         9
          11       1.00      0.11      0.19        19
          12       0.78      0.77      0.78      1129
          13       0.73      0.19      0.30        43
          14       0.77      0.15      0.25       158
          15       0.76      0.15      0.25        87
          16       0.95      0.55      0.70       392
          17       1.00    

In [54]:
metrics.average_precision_score(y_test_mlb, y_pred_clf,average='micro')

0.7858463816731243

recall score for  average type Micro

In [55]:
#metrics.recall_score(y_test_mlb, y_pred_clf)
metrics.recall_score(y_test_mlb, y_pred_clf, labels=None, pos_label=1, average='micro', sample_weight=None)

0.7913926297108258

recall score for  average type Macro

In [56]:

metrics.recall_score(y_test_mlb, y_pred_clf, labels=None, pos_label=1, average='macro', sample_weight=None)

C:\Users\pg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.47989830882639967

recall score for  average type weighted

In [57]:

metrics.recall_score(y_test_mlb, y_pred_clf, labels=None, pos_label=1, average='weighted', sample_weight=None)

0.7913926297108258

In [58]:
metrics.multilabel_confusion_matrix(y_test_mlb, y_pred_clf,sample_weight=None,labels=None,samplewise=False)

array([[[   0,    0],
        [   0, 2250]],

       [[2181,    5],
        [  52,   12]],

       [[2250,    0],
        [   0,    0]],

       [[1668,   40],
        [ 290,  252]],

       [[1663,   61],
        [ 341,  185]],

       [[ 752,  257],
        [ 216, 1025]],

       [[1903,   16],
        [ 213,  118]],

       [[1311,  142],
        [ 357,  440]],

       [[1721,   31],
        [ 262,  236]],

       [[1711,   34],
        [ 339,  166]],

       [[2240,    1],
        [   6,    3]],

       [[2231,    0],
        [  17,    2]],

       [[ 876,  245],
        [ 256,  873]],

       [[2204,    3],
        [  35,    8]],

       [[2085,    7],
        [ 134,   24]],

       [[2159,    4],
        [  74,   13]],

       [[1847,   11],
        [ 177,  215]],

       [[2217,    0],
        [  31,    2]],

       [[2199,    6],
        [  33,   12]],

       [[2042,   22],
        [ 145,   41]],

       [[2226,    0],
        [  23,    1]],

       [[2236,    0],
        [  1

Precision, recall and F1 score for Micro average

In [59]:
metrics.precision_recall_fscore_support(y_test_mlb, y_pred_clf, beta=1.0, average='micro', warn_for=('precision', 'recall', 'f-score'), sample_weight=None)

(0.9049750798340646, 0.7913926297108258, 0.8443813263160826, None)

Precision, recall and F1 score for Macro average

In [60]:
metrics.precision_recall_fscore_support(y_test_mlb, y_pred_clf, beta=1.0, average='macro', warn_for=('precision', 'recall', 'f-score'), sample_weight=None)

C:\Users\pg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\pg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


(0.7960378600957202, 0.47989830882639967, 0.5505062619573617, None)

Precision, recall and F1 score for Weighted average

In [61]:
metrics.precision_recall_fscore_support(y_test_mlb, y_pred_clf, beta=1.0, average='weighted', warn_for=('precision', 'recall', 'f-score'), sample_weight=None)

(0.8916613663525063, 0.7913926297108258, 0.8229279322663255, None)

In [65]:
y_pred_transform = mlb.inverse_transform(y_pred_clf)

In [66]:
y_test_transform = mlb.inverse_transform(y_test_mlb)

In [71]:
" ".join(y_pred_transform[1])

', 1 7 S U a c d e f i k l m n o p r'

In [72]:
" ".join(y_test_transform[1])

', 1 7 S U a c d e f i k l m n o p r'

In [73]:
for i in range(5):
    print("Predicted transform values is: -", "".join(y_pred_transform[i]))
    print("Actual transform values is : - ","".join(y_test_transform[i]))
    print("-----------------------------")

Predicted transform values is: - ,35ATaceghilmnorsy
Actual transform values is : -  ,35ATaceghilmnorsy
-----------------------------
Predicted transform values is: - ,17SUacdefiklmnopr
Actual transform values is : -  ,17SUacdefiklmnopr
-----------------------------
Predicted transform values is: - ,1SUadefiklmnopr
Actual transform values is : -  ,23CUacdefiklmnr
-----------------------------
Predicted transform values is: - ,Taeilmnrs
Actual transform values is : -  ,35ATaceghilmnorsy
-----------------------------
Predicted transform values is: - ,1Sadefilmnor
Actual transform values is : -  ,15ASadefilmnqrstu
-----------------------------
